In [1]:
from scipy.optimize import minimize
import numpy as np

In [2]:
def get_nu_star(xi, params):

    def objective(nu):

        if xi(nu, **params)>0:
            return -(np.log(xi(nu, **params)) - 0.5*np.dot(nu.T, nu))
        else:
            return 1e80
        
    nu0 = np.zeros(params["d"])

    result = minimize(objective, nu0, method='SLSQP')

    if result.success:
        return result.x
    else:
        raise ValueError("L'optimisation n'a pas réussi à converger.")

In [3]:
r = 0.05
T = 1
s0 = 50
sigma = 0.1
K = 50

B = 60

In [4]:
def asian_function(y, S0, v, mu, maturity, strike, rate, d):

    k = np.arange(1, d + 1)
    y_cum = np.cumsum(y)
    payoff = np.exp(-rate*maturity)*max(S0*np.sum(np.exp(mu*(k*T)/d + v*np.sqrt(T/d)*y_cum))/d - strike, 0)

    return payoff

In [5]:
d = 16
T = 1
r = 0.05

S0 = 50
K = 45
v = 0.1

params_asian = {
    "d": d,
    "S0": S0,
    "v": v,
    "mu": r-0.5*v**2,
    "maturity": T,
    "strike": K,
    "rate": r
}

In [6]:
nu = get_nu_star(asian_function, params_asian)/0.42
nu

array([0.41409411, 0.38929429, 0.36432272, 0.33918479, 0.31388597,
       0.28843211, 0.26282931, 0.23708353, 0.21120063, 0.18518756,
       0.15905015, 0.13279534, 0.10642983, 0.07996049, 0.05339445,
       0.02673861])

In [7]:
def knock_out_function(y, S0, v, mu, maturity, strike, rate, barrier, d):

    d = len(y)
    k = np.arange(1, d + 1)
    y_cum = np.cumsum(y)

    s = S0*np.exp(mu*maturity + v*np.sqrt(maturity/d)*y_cum[-1])
    payoff = np.exp(-rate*maturity)*max(S0*np.sum(np.exp(mu*(k*maturity)/d + v*np.sqrt(maturity/d)*y_cum))/d - strike, 0)*(1 if s <= barrier else 0)

    return payoff

In [8]:
d = 16
T = 1
r = 0.05

S0 = 50
K = 50
B = 60
v = 0.1

params_barrier = {
    "d": d,
    "S0": S0,
    "v": v,
    "mu": r-0.5*v**2,
    "maturity": T,
    "strike": K,
    "rate": r, 
    "barrier": B
}

In [9]:
nu = get_nu_star(knock_out_function, params_barrier)/0.84
nu

array([0.40948586, 0.38548032, 0.36121139, 0.33668898, 0.31192227,
       0.28692279, 0.26170077, 0.23626863, 0.21063744, 0.18482026,
       0.15882959, 0.13267844, 0.10638071, 0.07994953, 0.05339963,
       0.02674488])

In [10]:
def basket_options_function(y, S0, v, mu, sq_correl, maturity, strike, alpha, rate, d):

    y_tilde = np.dot(sq_correl, y)
    payoff = np.exp(-rate*maturity)*max(np.sum(alpha*S0*np.exp(mu*T + v*np.sqrt(maturity)*y_tilde)) - strike, 0)

    return payoff

In [11]:
d = 40
T = 1
r = 0.05

S0 = np.random.uniform(20, 80, d)
K = 45
alpha = np.ones(d)/d

c = 0.1
sigma = c*np.ones((d, d)) + (1-c)*np.eye(d)
v = np.linspace(0.1, 0.4, d)

params_basket = {
    "d": d,
    "S0": S0,
    "v": v,
    "mu": r-0.5*v**2,
    "sq_correl": np.linalg.cholesky(sigma),
    "maturity": T,
    "strike": K,
    "alpha": alpha,
    "rate": r
}

In [12]:
nu = get_nu_star(basket_options_function, params_basket)/0.41
nu

array([0.49387759, 0.45499311, 0.44233134, 0.42097589, 0.34820587,
       0.34137173, 0.30711638, 0.31763454, 0.29970626, 0.32864644,
       0.23891777, 0.24525889, 0.25001646, 0.23881122, 0.26902485,
       0.28386725, 0.17894612, 0.25946362, 0.22885432, 0.22403201,
       0.22220453, 0.24837555, 0.25918935, 0.12835982, 0.12469161,
       0.15650559, 0.25443503, 0.18985883, 0.12502427, 0.16651037,
       0.1740852 , 0.25243009, 0.16687629, 0.25858598, 0.12545466,
       0.16435488, 0.20683362, 0.11598998, 0.27572766, 0.24204319])